p and q are two large primes.
<br><br>
$n = p * q$
<br><br>
$|\mathbb{Z}_{n}^{*}| = \phi(n) = n * \prod_{\text{p: p is prime and p|n}} (1 - \frac{1}{p})$ (Euler’s phi function) $= n * (1 - \frac{1}{p}) * (1 - \frac{1}{q})$
<br><br>
$ = n * (1 - \frac{1}{q} - \frac{1}{p} + \frac{1}{p*q}) = p * q - p - q + 1$
<br><br>
$ = p * (q - 1) - (q - 1) = (p - 1)*(q - 1)$
<br><br>
$ gcd(e, \phi(n)) = 1$
<br><br>
$(e * d) \mod \phi(n) = 1$
<br><br>
$d \equiv e^{-1}\mod \phi(n)$ (Multiplicative Inverse of e modulo $\phi(n)$)
<br><br>
$ P = (e, n)$ (RSA public key)
<br><br>
$ S = (d, n)$ (RSA private key)
<br><br>
M is a message.
<br><br>
Sender uses user's public key/address to encrypt the message M: $P(M) = M^{e}(\mod n)$
<br><br>
User uses private key to decrypt the ciphertext: $S(P(M)) = P(M)^{d}(\mod n) \equiv M^{e * d}(\mod n)$
<br><br>
Claim: $S(P(M)) = M^{e * d}(\mod n) \equiv M(\mod n)$
<br><br>
Proof:
<br><br>
$ S(P(M)) = M^{e * d}(\mod n)$
<br><br>
$\because (e * d) \mod \phi(n) = 1$
<br><br>
$\therefore e * d = k * \phi(n) + 1 = k * (p - 1) *(q - 1) + 1$
<br><br>
If $ M \not \equiv 0(\mod p)$,
<br><br>
$ M^{e*d} \equiv M^{k * (p - 1)*(q - 1) + 1}(\mod p)$
<br><br>
$\equiv M * (M^{p - 1})^{k * (q - 1)})(\mod p)$
<br><br>
$\equiv M * (M^{p - 1}(\mod p))^{k * (q - 1)})(\mod p)$
<br><br>
$\equiv M * 1^{k * (q - 1)})(\mod p)$ (Fermat's Theorem)
<br><br>
$\equiv M(\mod p)$
<br><br>
If $ M \equiv 0(\mod p)$, then
<br><br>
$ M^{e*d}\equiv 0(\mod p) \equiv M(\mod p)$
<br><br>
$\therefore M^{e*d}(\mod p) \equiv M(\mod p)$, and similarly,
<br><br>
$M^{e*d}(\mod q) \equiv M(\mod q)$, and
<br><br>
$M^{e*d}(\mod n) \equiv M(\mod n)$ (Chinese remainder theorem)